<a href="https://colab.research.google.com/github/mk6749261-create/Named-Entity-Recognition/blob/main/Another_copy_of_notebook6e0d3a5a96.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
clmentbisaillon_fake_and_real_news_dataset_path = kagglehub.dataset_download('clmentbisaillon/fake-and-real-news-dataset')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd

In [ ]:
df1=pd.read_csv("/kaggle/input/fake-and-real-news-dataset/True.csv")
df2=pd.read_csv("/kaggle/input/fake-and-real-news-dataset/Fake.csv")

In [ ]:
df1.head()


In [ ]:
df1["subject"].value_counts()

In [ ]:
df1.columns

In [ ]:
df2.head()


In [ ]:
df2["subject"].value_counts()

In [ ]:
df1["label"]=1
df2["label"]=0

In [ ]:
df = pd.concat([df1, df2])
df

In [ ]:
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
df.head()

In [ ]:
df.isna().sum()

In [ ]:
df.duplicated()

In [ ]:
duplicated_rows=df[df.duplicated()]
(duplicated_rows)

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df.duplicated().sum()

In [ ]:
df["label"].value_counts()

In [ ]:
df.drop(["subject","date"],axis=1,inplace=True)

In [ ]:
df.head()

In [ ]:
df["content"]=df["title"]+" "+df["text"]

In [ ]:
df.head()

In [ ]:
df['content']=df['content'].str.lower()

In [ ]:
import string
df['content']=df['content'].str.translate(str.maketrans(" "," ",string.punctuation))

In [ ]:
df['content'].head()

In [ ]:
import nltk
nltk.download('punkt_tab')
df['content']=df['content'].apply(nltk.word_tokenize)

In [ ]:
df['content'].head()

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')


In [ ]:
stop_words=stopwords.words('english')
stop_words.remove('not')

In [ ]:
def remove_stopwords(text):
  return[word for word in text if word not in stop_words]
df['content']=df['content'].apply(remove_stopwords)

In [ ]:
df['content'].head()

In [ ]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')


In [ ]:
lemmatizer=WordNetLemmatizer()
def lemmatize_text(text):
  return[lemmatizer.lemmatize(word) for word in text]
df['content']=df['content'].apply(lemmatize_text)

In [ ]:
df['content'].head()

Integer Encoding before embedding

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [ ]:
# نحدد أقصى عدد كلمات في القاموس (vocab)
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(df['content'])

# القاموس الناتج
word_index = tokenizer.word_index
print(word_index)





In [ ]:
sequences=tokenizer.texts_to_sequences(df['content'])
print(sequences[0:5])

In [ ]:
padded=pad_sequences(sequences,maxlen=50,padding='post',truncating='post')
print(padded[0:5])

In [ ]:

print(padded.shape)
print(padded[0])

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(padded,df['label'],test_size=0.2)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

model = Sequential([
    Embedding(input_dim=10000, output_dim=64, input_length=200),
    LSTM(64, return_sequences=False),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


In [ ]:
history = model.fit(
    x_train, y_train,
    epochs=5,
    batch_size=64,
    validation_split=0.2,
    verbose=1
)

loss, acc = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {acc:.3f}")


In [ ]:
from sklearn.metrics import classification_report
y_pred=model.predict(x_test)
y_pred_classes=(y_pred>0.5).astype("int32")
print(classification_report(y_pred_classes,y_test))


In [ ]:
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay
cm=confusion_matrix(y_test,y_pred_classes)
print (cm)

In [ ]:
disply=ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=[False,True])
disply.plot(cmap="Blues", values_format="d")


In [ ]:
!pip install wordcloud


In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt


In [ ]:

real_text = " ".join(df[df['label'] == 1]['content'].apply(lambda x: " ".join(x)))


fake_text = " ".join(df[df['label'] == 0]['content'].apply(lambda x: " ".join(x)))

plt.figure(figsize=(12,6))
wc_real = WordCloud(width=800, height=400, background_color="white").generate(real_text)
plt.imshow(wc_real, interpolation="bilinear")
plt.title("Te Most Words Repeat in The Real News")
plt.axis("off")
plt.show()


plt.figure(figsize=(12,6))
wc_fake = WordCloud(width=800, height=400, background_color="white", colormap="Reds").generate(fake_text)
plt.imshow(wc_fake, interpolation="bilinear")
plt.title("The Most words repeat in The fack News")
plt.axis("off")
plt.show()

In [ ]:
sample = ["BREAKING: GOP Chairman Grassley Has Had Enoug"]
seq = tokenizer.texts_to_sequences(sample)
pad = pad_sequences(seq, maxlen=200, padding='post')
pred = model.predict(pad)
print("Real" if pred[0][0] >= 0.5 else "Fake")
